In [22]:
from typing import Union

import numpy as np
import torch
from colour import Color
from human_body_prior.body_model.body_model import BodyModel
from torch import nn

from human_body_prior.models.ik_engine import IK_Engine
from os import path as osp


In [23]:
class SourceKeyPoints(nn.Module):
    def __init__(self,
                 bm: Union[str, BodyModel],
                 n_joints: int=24,
                 ):
        super(SourceKeyPoints, self).__init__()
        
        self.bm = BodyModel(bm, persistant_buffer=False) if isinstance(bm, str) else bm
        self.bm_f = []
        self.n_joints = n_joints
        self.kpts_colors = np.array([Color('grey').rgb for _ in range(n_joints)])
        
    def forward(self, body_parms):
        new_body = self.bm(**body_parms)
        
        return {'source_kpts': new_body.Jtr[:, :self.n_joints], 'body': new_body,
                'params': body_parms}

In [24]:
support_dir = '../support_data/dowloads'
vposer_expr_dir = osp.join(support_dir,'vposer_v2_05')
bm_fname =  osp.join(support_dir,'/home/yeyiqi/Documents/models/SMPLX/models_smplx_v1_1/models/smplx/SMPLX_NEUTRAL.npz')
n_joints = 24
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
data_loss = torch.nn.MSELoss(reduction='sum')
stepwise_weights = [
    {
        'data':10,
        'Poz_body':.01,
        'betas':.5
    }
]
optimizer_args = {
    'type':'LBFGS',
    'max_iter':300,
    'lr':1,
    'tolerance_change':1e-4,
    'history_size':200
}
ik_engine = IK_Engine(
    vposer_expr_dir=vposer_expr_dir,
    verbosity=2,
    display_rc=(2, 2),
    data_loss=data_loss,
    stepwise_weights=stepwise_weights,
    optimizer_args=optimizer_args
).to(comp_device)

2024-12-17 11:13:57.618 | INFO     | human_body_prior.tools.model_loader:load_model:97 - Loaded model in eval mode with trained weights: ../support_data/dowloads/vposer_v2_05/snapshots/vposer_v2_05.ckpt


In [26]:
source_pts = SourceKeyPoints(bm=bm_fname, n_joints=n_joints).to(comp_device)

In [27]:
source_pts

SourceKeyPoints(
  (bm): BodyModel()
)

In [28]:
sample_amass = {
    'poses':torch.zeros(1, 66).type(torch.float),
    'trans':torch.zeros(1, 3).type(torch.float)
}

In [29]:
target_bm = BodyModel(bm_fname)(**{
    'pose_body': torch.tensor(sample_amass['poses'][:,3:66]).type(torch.float),
    'root_orient': torch.tensor(sample_amass['poses'][:,:3]).type(torch.float),
    'trans': torch.tensor(sample_amass['trans']).type(torch.float)
})

/home/yeyiqi/.conda/envs/Vposer/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/yeyiqi/.conda/envs/Vposer/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yeyiqi/.conda/envs/Vposer/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [30]:
target_bm.Jtr

tensor([[[ 3.1233e-03, -3.5141e-01,  1.2037e-02],
         [ 6.1313e-02, -4.4417e-01, -1.3965e-02],
         [-6.0144e-02, -4.5532e-01, -9.2138e-03],
         [ 3.6057e-04, -2.4152e-01, -1.5581e-02],
         [ 1.1601e-01, -8.2292e-01, -2.3361e-02],
         [-1.0435e-01, -8.1770e-01, -2.6038e-02],
         [ 9.8083e-03, -1.0966e-01, -2.1521e-02],
         [ 7.2555e-02, -1.2260e+00, -5.5237e-02],
         [-8.8937e-02, -1.2284e+00, -4.6230e-02],
         [-1.5222e-03, -5.7428e-02,  6.9258e-03],
         [ 1.1981e-01, -1.2840e+00,  6.2980e-02],
         [-1.2775e-01, -1.2868e+00,  7.2819e-02],
         [-1.3687e-02,  1.0774e-01, -2.4690e-02],
         [ 4.4842e-02,  2.7515e-02, -2.9465e-04],
         [-4.9217e-02,  2.6910e-02, -6.4741e-03],
         [ 1.1097e-02,  2.6819e-01, -3.9522e-03],
         [ 1.6408e-01,  8.5243e-02, -1.5756e-02],
         [-1.5179e-01,  8.0435e-02, -1.9143e-02],
         [ 4.1820e-01,  1.3093e-02, -5.8214e-02],
         [-4.2294e-01,  4.3942e-02, -4.5610e-02],


In [31]:
source_pts = SourceKeyPoints(bm=bm_fname, n_joints=24).to(comp_device)
target_pts = target_bm.Jtr[:, :n_joints].detach().to(comp_device)
ik_res = ik_engine(source_pts, target_pts)

it 1 -- [total loss = 4.11e-02] - data = 4.11e-02 | betas = 1.19e-05
it 2 -- [total loss = 4.84e+01] - data = 4.84e+01 | betas = 5.13e-05
it 3 -- [total loss = 3.67e-02] - data = 3.67e-02 | betas = 1.22e-05
it 4 -- [total loss = 3.58e-02] - data = 3.58e-02 | betas = 1.27e-05
it 5 -- [total loss = 2.92e-02] - data = 2.92e-02 | betas = 1.84e-05
it 6 -- [total loss = 2.68e-02] - data = 2.68e-02 | betas = 3.75e-05
it 7 -- [total loss = 2.38e-02] - data = 2.38e-02 | betas = 3.90e-05
it 8 -- [total loss = 2.31e-02] - data = 2.31e-02 | betas = 3.75e-05
it 9 -- [total loss = 2.28e-02] - data = 2.28e-02 | betas = 3.83e-05


In [32]:
ik_res_detached = {k: v.detach() for k, v in ik_res.items()}

In [33]:
ik_res_detached

{'trans': tensor([[ 0.0022, -0.0002, -0.0027]], device='cuda:0'),
 'betas': tensor([[ 8.4177e-03, -1.7537e-03, -2.2522e-04,  7.1737e-04, -1.3587e-03,
          -1.1679e-04,  8.3314e-05,  4.1969e-04, -2.4188e-05,  2.1266e-04]],
        device='cuda:0'),
 'root_orient': tensor([[-0.0202,  0.0007, -0.0023]], device='cuda:0'),
 'poZ_body': tensor([[ 8.6341e-01, -1.7323e-01, -2.2442e-01, -4.9408e-01,  4.3026e-01,
           3.4344e-01,  4.8484e-01,  3.3179e-01, -2.6103e-01, -1.5467e-03,
          -3.3897e-01, -4.5619e-01,  2.6115e-01,  1.8094e-01,  5.0699e-04,
           2.2413e-01,  5.6722e-03, -9.9848e-02,  4.7442e-01, -2.3238e+00,
          -1.0477e+00, -4.8793e-01, -8.4189e-02, -1.2253e-03,  3.8270e-03,
          -8.2918e-01, -4.2611e-02, -4.6918e-01, -2.5755e-02, -2.9061e-01,
          -3.1770e-01,  2.9578e-03]], device='cuda:0'),
 'pose_body': tensor([[ 2.9855e-02,  3.5898e-02,  3.7896e-03, -4.1665e-02, -4.6016e-02,
           2.8517e-04,  1.1198e-01,  1.4137e-02,  1.6627e-02, -1.0851

In [14]:
nan_mask = torch.isnan(ik_res_detached['trans']).sum(-1) != 0

In [15]:
if nan_mask.sum() != 0: raise ValueError('Sum results were NaN!')

In [16]:
nan_mask

tensor([False, False, False, False], device='cuda:0')

In [19]:
target_pts.shape

torch.Size([1, 24, 3])

In [17]:
res = source_pts.bm.forward()

In [21]:
res.full_pose

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0')